In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls ./gdrive/'My Drive'/"Colab Notebooks"/cifar10

t_train.npy  x_test.npy  x_train.npy


In [0]:
import numpy as np
import pandas as pd
import os
def load_cifar10():
    data_dir = './gdrive/My Drive/Colab Notebooks/cifar10/'
    # 学習データ
    x_train = np.load(os.path.join(data_dir,'x_train.npy'))
    t_train = np.load(os.path.join(data_dir,'t_train.npy'))

    # テストデータ
    x_test = np.load(os.path.join(data_dir,'x_test.npy'))
    
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    
    t_train = np.eye(10)[t_train.astype('int32').flatten()]
    
    return (x_train, x_test, t_train)

In [0]:
#%%writefile /root/userspace/submission_code.py
def main():
  import tensorflow as tf
  from sklearn.utils import shuffle
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score

  rng = np.random.RandomState(1234)
  random_state = 42

  def tf_log(x):
      # WRITE ME
      return tf.log(tf.clip_by_value(x, 1e-10, x))
  ### ネットワーク ###
  tf.reset_default_graph()
  is_training = tf.placeholder(tf.bool, shape=())

  # WRITE ME
  ###################################################################################

  x = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32)
  t = tf.placeholder(tf.float32, [None, 10])

  ### ネットワーク ###
  h = tf.layers.Conv2D(filters=32, kernel_size= [3, 3])(x) # 32x32x3 -> 30x30x32 # conv2dの初期設定: strides=(1, 1), padding='valid' 
  h = tf.layers.BatchNormalization()(h, training=is_training)
  h = tf.nn.relu(h)
  h = tf.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) # 30x30x32 -> 15x15x32

  h = tf.layers.Conv2D(filters=64, kernel_size= [3, 3])(h) # 15x15x32 -> 13x13x64
  h = tf.layers.BatchNormalization()(h, training=is_training)
  h = tf.nn.relu(h)
  h = tf.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) # 13x13x64 -> 6x6x64

  h = tf.layers.Conv2D(filters=128, kernel_size= [3, 3])(h) # 6x6x64 -> 4x4x128
  h = tf.layers.BatchNormalization()(h, training=is_training)
  h = tf.nn.relu(h)
  h = tf.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) # 4x4x128 -> 2x2x128

  h = tf.layers.Flatten()(h)
  h = tf.layers.Dense(units=256, activation=tf.nn.relu)(h)
  y = tf.layers.Dense(units=10, activation=tf.nn.softmax)(h) # WRITE ME

  cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1))
  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

  with tf.control_dependencies(update_ops):
      optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

  ###################################################################################


  ### 前処理 ###
  def gcn(x):
      # WRITE ME
      mean = np.mean(x, axis=(1, 2, 3), keepdims=True)
      std = np.std(x, axis=(1, 2, 3), keepdims=True)
      return (x - mean)/std

  class ZCAWhitening:
      # WRITE ME
      def __init__(self, epsilon=1e-4):
          self.epsilon = epsilon
          self.mean = None
          self.ZCA_matrix = None

      def fit(self, x):
          x = x.reshape(x.shape[0], -1)
          self.mean = np.mean(x, axis=0)
          x -= self.mean
          cov_matrix = np.dot(x.T, x) / x.shape[0]
          A, d, _ = np.linalg.svd(cov_matrix)
          self.ZCA_matrix = np.dot(np.dot(A, np.diag(1. / np.sqrt(d + self.epsilon))), A.T)

      def transform(self, x):
          shape = x.shape
          x = x.reshape(x.shape[0], -1)
          x -= self.mean
          x = np.dot(x, self.ZCA_matrix.T)
          return x.reshape(shape)

  x_train, x_test, t_train = load_cifar10()
  x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)
  zca = ZCAWhitening()
  zca.fit(x_train)
  x_train_zca = zca.transform(gcn(x_train))
  t_train_zca = t_train[:]
  x_valid_zca = zca.transform(gcn(x_valid))
  t_valid_zca = t_valid[:]
  x_test_zca = zca.transform(gcn(x_test))

  ### 学習 ###
  n_epochs = 10
  batch_size = 100
  n_batches = x_train.shape[0]//batch_size

  sess = tf.Session()
  init = tf.global_variables_initializer()
  sess.run(init)

  for epoch in range(n_epochs):
      # WRITE ME
      x_train_zca, t_train_zca = shuffle(x_train_zca, t_train_zca, random_state=random_state)
      for batch in range(n_batches):
          start = batch * batch_size
          end = start + batch_size
          sess.run(optimizer, feed_dict={x: x_train_zca[start:end], t: t_train_zca[start:end], is_training: True})
      y_pred, cost_valid = sess.run([y, cost], feed_dict={x: x_valid_zca, t: t_valid_zca, is_training: False})
      print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
          epoch,
          cost_valid,
          accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
      ))
  sess.close()

  y_pred = None # WRITE ME
  submission = pd.Series(y_pred, name='label')
  #submission.to_csv('/root/userspace/submission_pred.csv', header=True, index_label='id')

In [0]:
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import os
def build_model(input_shape, num_classes):
#   source = tf.keras.Input(name="seed", shape=input_shape, batch_size=batch_size)
  source = tf.keras.Input(name="seed", shape=input_shape)
  x = tf.keras.layers.Convolution2D(192,(5,5),strides=(1,1),padding='same',activation='relu')(source)
  x = tf.keras.layers.Convolution2D(160,(1,1),strides=(1,1),padding='same',activation='relu')(x)
  x = tf.keras.layers.Convolution2D(96,(1,1),strides=(1,1),padding='same',activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPool2D((3,3),strides=(2,2),padding='same')(x)          
  x = tf.keras.layers.Dropout(0.5)(x)

  x = tf.keras.layers.Convolution2D(192,(5,5),strides=(1,1),padding='same',activation='relu')(x)
  x = tf.keras.layers.Convolution2D(192,(1,1),strides=(1,1),padding='same',activation='relu')(x)
  x = tf.keras.layers.Convolution2D(192,(1,1),strides=(1,1),padding='same',activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.AvgPool2D((3,3),strides=(2,2),padding='same')(x)          
  x = tf.keras.layers.Dropout(0.5)(x)

  x = tf.keras.layers.Convolution2D(192,(3,3),strides=(1,1),padding='same',activation='relu')(x)
  x = tf.keras.layers.Convolution2D(192,(1,1),strides=(1,1),padding='same',activation='relu')(x)
  x = tf.keras.layers.Convolution2D(num_classes,(1,1),1,padding='same',activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.AvgPool2D((8,8),1,padding='valid')(x)
  x = tf.keras.layers.Flatten()(x)
  model = tf.keras.Model(inputs=[source], outputs=[x])
  return model

def get_data_generator(batch_size):
  rng = np.random.RandomState(1234)
  random_state = 42
  
  x_train, x_test, t_train = load_cifar10()
  x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)
  
  datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
  
  # compute quantities required for featurewise normalization
  # (std, mean, and principal components if ZCA whitening is applied)
  datagen.fit(x_train)

def to_tpu(model):
  tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
  strategy = tf.contrib.tpu.TPUDistributionStrategy(tpu_cluster_resolver)
  model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
  return model

def save_history(history):
  history_dir = './gdrive/My Drive/Colab Notebooks/history'
  with open(os.path.join(history_dir,'cifar10.dat'), "wb") as fp:
    pickle.dump(history, fp)
  
def train():
  rng = np.random.RandomState(1234)
  random_state = 42
  x_train, x_test, t_train = load_cifar10()
  x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)
    
  
  batch_size=2048#256
  learning_rate=0.01
  nb_epoch = 10
  model = build_model((32,32,3),10)
  print(model.summary())
  model.compile(optimizer=tf.keras.optimizers .Adam(lr=learning_rate), loss="categorical_crossentropy", metrics=["accuracy"])
  model = to_tpu(model)
  
  
  datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zca_whitening=False
  )  
  datagen.fit(x_train)  
  history = model.fit(x_train, t_train, batch_size=batch_size, epochs=nb_epoch, validation_data=(x_valid, t_valid)).history
  model.fit_generator(datagen.flow(x_train, t_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, epochs=nb_epoch,
                     validation_data=(x_valid, t_valid)
                     )
  for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, t_batch in datagen.flow(x_train, t_train, batch_size=batch_size):
        history=model.fit(x_batch, t_batch).history
        batches += 1
        if batches >= len(x_train) / batch_size:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    save_history(history)
train()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
seed (InputLayer)            (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 32, 32, 192)       14592     
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 32, 32, 160)       30880     
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 32, 32, 96)        15456     
_________________________________________________________________
batch_normalization_27 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 16, 16, 96)        0         
__________